In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium
import plotly.express as px
import geopandas as gpd
import matplotlib.patches as mpatches

### Setting Up Data and Some Measures

#### Import Stations, Define Haversine

In [4]:
stations = pd.read_csv('11.17.25_Bluebikes_Station_List.csv', skiprows=1)
print(stations.head())

# Define the given latitude and longitude (center point)
center_lat = stations['Lat'].mean()
center_lon = stations['Long'].mean()

# Function to calculate distance using the Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])  # Convert degrees to radians
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

# Calculate distance to the center for each row
stations['distance_to_center'] = haversine(stations['Lat'], stations['Long'], center_lat, center_lon)
stations['far_from_center'] = stations['distance_to_center'] > 4


   Number                               NAME        Lat       Long  \
0  C32083  Washington St at Walsh Playground  42.277195 -71.069556   
1  C32067       Washington St at Peters Park  42.343852 -71.067646   
2  C32084         Washington St at Fuller St  42.281986 -71.071479   
3  C32081       Washington St at Denton Terr  42.280728 -71.134238   
4  C32042        Washington St at Bowdoin St  42.299165 -71.073459   

  Seasonal Status Municipality  Total Docks  \
0  Winter Storage       Boston           19   
1  Winter Storage       Boston           19   
2      Year Round       Boston           15   
3  Winter Storage       Boston           19   
4  Winter Storage       Boston           15   

  Station ID (to match to historic system data)  
0                                           430  
1                                           412  
2                                           431  
3                                           427  
4                                           25

#### Import Trip Data

In [5]:
oct25trips = pd.read_csv('202510-bluebikes-tripdata.csv')
oct25started = oct25trips.groupby('end_station_id').describe()['start_lat']
oct25started = oct25started.sort_values(by='count')

ebikes = oct25trips[oct25trips['rideable_type'] == 'electric_bike']

ebikes_by_endpoint = ebikes.groupby('end_station_id').describe()['end_lat']
ebikes_by_endpoint = ebikes_by_endpoint.sort_values(by='count')

#plt.bar(ebikes_by_endpoint.index, ebikes_by_endpoint['count'])
#plt.title('E-bike rides by end station')
#plt.show()

#plt.bar(oct25started.index,oct25started['count'])
#plt.title('Rides by end station')
#plt.show()

#### Add to Stations: Count of All Bikes, Count of E-Bikes, E-Bike Trip Ratio

In [6]:
stations = stations.merge(
    oct25started[['count']],
    how='left',  # Use 'left' join to keep all rows in stations
    left_on='Number',  # Column in stations to join on
    right_on=oct25started.index  # Column in oct25started to join on
)

stations = stations.merge(
    ebikes_by_endpoint[['count']],
    how='left',  # Use 'left' join to keep all rows in stations
    left_on='Number',  # Column in stations to join on
    right_on=ebikes_by_endpoint.index  # Column in oct25started to join on
)

stations = stations.rename(columns = {'count_x': 'count_all_bikes', 'count_y': 'count_ebikes'} )

# Ensure the columns are Series, not DataFrames
stations['count_all_bikes'] = stations['count_all_bikes'].squeeze()
stations['count_ebikes'] = stations['count_ebikes'].squeeze()

stations['ratio_ebikes'] = stations['count_ebikes'] / stations['count_all_bikes'].replace(0, np.nan)

#### Count of Total Starts and Ends by Station

In [7]:
# Count of Total Starts and Ends by Station

ebikes_by_start = ebikes.groupby(['start_station_id']).size().reset_index(name='num_starts')
ebikes_by_start = ebikes_by_start.sort_values(by='num_starts', ascending=False)

ebikes_by_end = ebikes.groupby(['end_station_id']).size().reset_index(name='num_ends')
ebikes_by_end = ebikes_by_end.sort_values(by='num_ends', ascending=False)

station_lookup = stations.set_index('Number')['NAME']
ebikes_by_start['start_station_name'] = ebikes_by_start['start_station_id'].map(station_lookup)
ebikes_by_end['end_station_name'] = ebikes_by_end['end_station_id'].map(station_lookup)

#### Adding Proportion of Member Rides (by end station) per Station

In [8]:
member_rides = (
    ebikes[ebikes['member_casual'] == 'member']
    .groupby('end_station_id')
    .size()
    .reset_index(name='member_rides')
)

station_member_prop = (
    ebikes_by_end
    .merge(
        member_rides,
        on='end_station_id',
        how='left')
)

station_member_prop[['num_ends', 'member_rides']] = (station_member_prop[['num_ends', 'member_rides']].fillna(0))
station_member_prop['member_prop'] = station_member_prop['member_rides'] / station_member_prop['num_ends']
station_member_prop = station_member_prop.rename(columns={'num_ends': 'total_rides'})
station_member_prop = station_member_prop[['end_station_id','end_station_name','total_rides','member_rides','member_prop']]

#### Starts per Dock per Station

In [9]:
# Number of Departures per Dock per Station

docks_map = stations.set_index('Number')['Total Docks']

ebikes_by_start['starts_per_dock'] = ebikes_by_start['start_station_id'].map(docks_map)

ebikes_by_start['starts_per_dock'] = ebikes_by_start['num_starts'] / ebikes_by_start['starts_per_dock']

ebikes_by_start = ebikes_by_start[np.isfinite(ebikes_by_start['starts_per_dock'])]
ebikes_by_start.sort_values(by='starts_per_dock', ascending=False)

,start_station_id,num_starts,start_station_name,starts_per_dock
2,A32002,1558,Commonwealth Ave at Agganis Way,103.866667
405,M32018,1380,Harvard Square at Mass Ave/ Dunster,72.631579
117,B32065,1674,Massachusetts Ave at Boylston St.,69.750000
114,B32062,1037,Forsyth St at Huntington Ave,69.133333
398,M32011,1199,Central Square at Mass Ave / Essex St,63.105263
...,...,...,...,...
501,R32007,4,Revere Public Library,0.363636
88,B32024,6,Pilgrim Rd at Brookline Ave,0.315789
224,C32100,5,Hyde Park Ave at Arlington St,0.263158
335,E32014,5,Ross Playground,0.263158


#### Quantile Measures of Departures

In [10]:
ebikes_by_start['departure_percentile'] = (ebikes_by_start['num_starts'].rank(pct=True) * 100)  # stores the station's quantile for departures

departure_quantiles = ebikes_by_start['num_starts'].quantile([0.25, 0.5, 0.75])  # returns the 25th, 50th, and 75th quartiles
departure_quantiles

0.25     73.0
0.50    189.0
0.75    336.0
Name: num_starts, dtype: float64

#### Frequency of Each Unique Trip

In [11]:
# Frequency of Each Unique Route (station i to station j)

ebikes_by_trip = ebikes.groupby(['start_station_id','end_station_id']).size().reset_index(name='num_trips')
ebikes_by_trip = ebikes_by_trip.sort_values(by='num_trips', ascending=False)

station_lookup = stations.set_index('Number')['NAME']  # Set dictionary for map func to reference

ebikes_by_trip['start_station_name'] = ebikes_by_trip['start_station_id'].map(station_lookup)  # Names of start and end stations
ebikes_by_trip['end_station_name'] = ebikes_by_trip['end_station_id'].map(station_lookup)

ebikes_by_trip

,start_station_id,end_station_id,num_trips,start_station_name,end_station_name
340,A32002,B32060,136,Commonwealth Ave at Agganis Way,Commonwealth Ave at Granby St
12169,B32060,A32002,130,Commonwealth Ave at Granby St,Commonwealth Ave at Agganis Way
12744,B32063,B32062,107,St. Alphonsus St at Tremont St,Forsyth St at Huntington Ave
398,A32002,D32032,90,Commonwealth Ave at Agganis Way,Silber Way
8956,B32016,M32006,89,Beacon St at Massachusetts Ave,MIT at Mass Ave / Amherst St
...,...,...,...,...,...
53461,Z32998,M32063,1,Broadway at Kittie Knox Bike Path,Sennott Park Broadway at Norfolk Street
23123,D32006,M32037,1,Lewis Wharf at Atlantic Ave,Ames St at Main St
53459,Z32998,M32061,1,Broadway at Kittie Knox Bike Path,Mass Ave at Albany St
23122,D32006,M32034,1,Lewis Wharf at Atlantic Ave,EF - North Point Park


In [12]:
# Table with number of stations that bikes come from and number of stations that bikes go to
unique_sources = (ebikes_by_trip.groupby('end_station_id').size().reset_index(name='num_unique_sources'))
unique_sources['end_station_name'] = (unique_sources['end_station_id'].map(station_lookup))

unique_destinations = (ebikes_by_trip.groupby('start_station_id').size().reset_index(name='num_unique_destinations'))
unique_destinations['start_station_name'] = (unique_destinations['start_station_id'].map(station_lookup))

station_stats = (unique_destinations.merge(unique_sources, left_on='start_station_id', right_on='end_station_id',how='outer'))

station_stats['station_id'] = (station_stats['start_station_id'].combine_first(station_stats['end_station_id']))
station_stats['station_name'] = station_stats['station_id'].map(station_lookup)

station_stats = station_stats[['station_id', 'station_name', 'num_unique_destinations', 'num_unique_sources']]

station_stats['diff'] = np.abs(station_stats['num_unique_destinations'] - station_stats['num_unique_sources'])

station_stats = station_stats.sort_values(by='diff', ascending=False)
station_stats.head(20)


,station_id,station_name,num_unique_destinations,num_unique_sources,diff
118,B32067,Commonwealth Ave at Blandford Mall,136,95,41
9,A32010,South Station - 700 Atlantic Ave,220,258,38
53,A32058,Tremont St at Court St,146,112,34
426,M32040,University Park,151,119,32
257,D32016,Charles Circle - Charles St at Cambridge St,231,263,32
368,K32002,Beacon St at Tappan St,96,124,28
96,B32032,Government Center - Cambridge St at Court St,234,206,28
289,D32049,Stuart St at Berkeley St,193,165,28
530,S32039,Elm St at White St,131,158,27
440,M32054,699 Mt Auburn St,95,121,26


#### Adding Trip Length

In [13]:
ebikes['started_at'] = pd.to_datetime(ebikes['started_at'])
ebikes['ended_at'] = pd.to_datetime(ebikes['ended_at'])

time_delta = ebikes['ended_at'] - ebikes['started_at']
ebikes['minutes_elapsed'] = time_delta.dt.total_seconds() / 60

C:\Users\Julie\AppData\Local\Temp\ipykernel_16652\2018164369.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ebikes['started_at'] = pd.to_datetime(ebikes['started_at'])
C:\Users\Julie\AppData\Local\Temp\ipykernel_16652\2018164369.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ebikes['ended_at'] = pd.to_datetime(ebikes['ended_at'])
C:\Users\Julie\AppData\Local\Temp\ipykernel_16652\2018164369.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

### Maps with Folium

#### Function to Get Map of Stations

In [14]:
from matplotlib.colors import to_hex
import branca.colormap as colm  

# Create a colormap using branca that matches the matplotlib colormap
colormap = colm.LinearColormap(
    colors=[to_hex(plt.get_cmap('cool')(i)) for i in range(256)],
    vmin=stations['ratio_ebikes'].min(),
    vmax=stations['ratio_ebikes'].max(),
    caption='Ratio of E-Bikes'  # Add a caption for the color bar
)

def build_map(id_list):  #list = ['id1', 'id2', 'id3', ...]

    # Create a folium map centered at the average latitude and longitude
    station_map = folium.Map(location=[center_lat, center_lon], zoom_start=13)
    threshold_red = 0.3
    cm = plt.get_cmap('cool')

    for id in id_list:
        row = stations.loc[stations['Number'] == id]

        folium.CircleMarker(
            location=[row['Lat'].iloc[0], row['Long'].iloc[0]],
            color= 'blue', 
            fill=True,
            fill_color= 'blue', 
            fill_opacity=0.6,
            popup=f"Name: {row['NAME']}, e-bike count: {row['count_ebikes']}"  # Optional popup with station name
        ).add_to(station_map)
    
    # Add the colormap to the map
    colormap.add_to(station_map)

    display(station_map)

### Max Min Distance

This fairness metric will determine which $k$ stations out of $n$ total stations to electrify such that we minimize the maximum of minimum distance from any station $x_i$ to an electrified station $x_j$: $$ \max_{i \in n} \left( \min_{k \in j} d(x_i, x_j) \right) $$

#### Open to set station coordinates and distance matrix (choose to include/exclude Salem)

##### Includes all $n$ stations in matrix

In [61]:
# Import to generate combinations of possible k stations
from itertools import combinations

# Set station coordinates
coordinates = stations.set_index('Number')[['Lat', 'Long']].to_dict('index')
station_ids = coordinates.keys()

# Precompute distance matrix
lat = stations['Lat'].values
lon = stations['Long'].values
coordinates = np.column_stack((lat, lon))

lat1 = coordinates[:, None, 0]
lat2 = coordinates[None, :, 0]
lon1 = coordinates[:, None, 1]
lon2 = coordinates[None, :, 1]
dist_matrix = haversine(lat1, lon1, lat2, lon2)

# Map station ID to index in matrix
id_to_idx = {station_id: idx for idx, station_id in enumerate(station_ids)}

##### Excludes Salem stations from matrix

In [16]:
# Set station coordinates
stations_filtered = stations[stations['Municipality'] != 'Salem'].copy()
coordinates = stations_filtered.set_index('Number')[['Lat', 'Long']].to_dict('index')
station_ids = coordinates.keys()

# Precompute Distance Matrix
lat = stations_filtered['Lat'].values
lon = stations_filtered['Long'].values
coordinates = np.column_stack((lat, lon))

lat1 = coordinates[:, None, 0]
lat2 = coordinates[None, :, 0]
lon1 = coordinates[:, None, 1]
lon2 = coordinates[None, :, 1]
dist_matrix = haversine(lat1, lon1, lat2, lon2)

# Map station ID to index in matrix
id_to_idx = {station_id: idx for idx, station_id in enumerate(station_ids)}

#### Max Min Function -- Input: stations list and $k$, Output: best $k$ stations and final distance

In [17]:
def maxmin_k_stations(bike_stations, k):                          # bike_stations = list of station IDs as strings ['id1','id2',...], k = number included in final rankings
    
    combos = combinations(bike_stations, k)

    final_min_max = 100000000000
    max_combo = []

    for combo in combos:
        combo_idx = [id_to_idx[ID] for ID in combo]        # Convert station IDs to indices in distance matrix
        min_dists = dist_matrix[:, combo_idx].min(axis=1)  # Get distance to nearest station in combo for all stations
        max_dist = min_dists.max()                         # Max distance from any station to nearest in combo
        
        if max_dist < final_min_max:
            final_min_max = max_dist
            max_combo = combo
    
    print("Max Distance:", final_min_max)
    print("--Stations List--")
    for i in range(len(max_combo)):
        station = stations.set_index('Number').loc[max_combo[i], 'NAME']
        print(station)

    return final_min_max, max_combo

### Metric: Charging Time Score

#### Score Description

The goal of this metric is to determine which stations balance high daytime usage with low nighttime usage/longer low usage blocks to allow for adequate charging windows (i.e. if the busiest station in Boston has bikes leaving every 5 minutes throughout the day, will the charging actually be useful?)
$$N_i(t,\Delta) = \text{the number of departures from station } i \text{ in the time window } [t, t+\Delta)$$
where $\Delta$ is a time interval amount to bucket the data. Then, let $\tau$ be a threshold for the number of departures in the window considered to be efficient (consider using the nth percentile of the top m stations as the threshold?) Then, we can say that the total charging time available at station $i$ in the day is: $$C_i = \sum_t \mathbb{1} \{N_i(t,\Delta) \leq \tau\} \cdot \Delta$$ 
where the indicator function is equal to 1 if the number of departures in the window is below the threshold $\tau$.\

The least used stations have the highest values for $C_i$, so instead of using the total number of charging minutes, the code above produces a charging score that considers both available minutes for charging and total number of departures over the month with the measure $$\text{charging score} = \text{number of departures} \cdot \frac{\text{number of downtime 30min intervals}}{\text{total number of intervals}}$$
The charging score should be highest when both downtime and number of departures are high (should consider that significantly higher numbers of departures skew results)

#### Finding Quantiles -- measure which stations have more concentrated bike use by time distribution

In [18]:
ebikes['start_minutes'] = ebikes['started_at'].dt.hour * 60 + ebikes['started_at'].dt.minute

station_quantiles = ebikes.groupby('start_station_id')['start_minutes'].quantile([0.25, 0.75]).unstack()
station_quantiles = station_quantiles.rename(columns={0.25: 'q25', 0.75: 'q75'})

station_quantiles['quantile_difference'] = station_quantiles['q75'] - station_quantiles['q25']  # Difference between quantiles
station_quantiles = station_quantiles.sort_values(by='quantile_difference')

station_quantiles['station_name'] = station_quantiles.index.map(station_lookup)  # Adding station name
station_quantiles = station_quantiles[['station_name', 'q25', 'q75', 'quantile_difference']]  # Reorder columns

# Lower number -> more concentrated bike use
#station_quantiles.head()

C:\Users\Julie\AppData\Local\Temp\ipykernel_16652\1408650780.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ebikes['start_minutes'] = ebikes['started_at'].dt.hour * 60 + ebikes['started_at'].dt.minute


#### Function to Get Station Departure Time Quantiles

In [19]:
def get_station_quantiles(e_bikes):
    e_bikes['start_minutes'] = e_bikes['started_at'].dt.hour * 60 + e_bikes['started_at'].dt.minute

    station_quantiles = e_bikes.groupby('start_station_id')['start_minutes'].quantile([0.25, 0.75]).unstack()
    station_quantiles = station_quantiles.rename(columns={0.25: 'q25', 0.75: 'q75'})

    station_quantiles['quantile_difference'] = station_quantiles['q75'] - station_quantiles['q25']  # Difference between quantiles
    station_quantiles = station_quantiles.sort_values(by='quantile_difference')

    station_quantiles['station_name'] = station_quantiles.index.map(station_lookup)  # Adding station name
    station_quantiles = station_quantiles[['station_name', 'q25', 'q75', 'quantile_difference']]  # Reorder columns

    return station_quantiles

#### Function to Compute the Downtime Fraction for delta, tau values

In [20]:
def compute_downtime_fraction(e_bikes, delta, tau, station_lookup):
    e_bikes = e_bikes.copy()
    e_bikes['time_bucket'] = e_bikes['started_at'].dt.floor(delta)

    departures = (e_bikes.groupby(['start_station_id', 'time_bucket']).size().reset_index(name='num_departures'))

    full_range = pd.date_range(
        start=e_bikes['started_at'].min().floor(delta),
        end=e_bikes['started_at'].max().ceil(delta),
        freq=delta
    )

    bikestations = e_bikes['start_station_id'].unique()
    full_index = pd.MultiIndex.from_product([bikestations, full_range], names=['start_station_id', 'time_bucket'])

    departures = (departures.set_index(['start_station_id', 'time_bucket']).reindex(full_index, fill_value=0).reset_index())
    departures['is_downtime'] = departures['num_departures'] <= tau

    downtime_fraction = (departures.groupby('start_station_id')['is_downtime'].mean().rename('downtime_fraction').reset_index())
    downtime_fraction['station_name'] = (downtime_fraction['start_station_id'].map(station_lookup))

    downtime_fraction['delta'] = delta
    downtime_fraction['tau'] = tau

    return downtime_fraction

#### Function to Get Station Departure Info

In [21]:
def get_departure_info(e_bikes):
    station_departures = e_bikes.groupby(['start_station_id']).size().reset_index(name='num_starts')
    station_departures = station_departures.sort_values(by='num_starts', ascending=False)

    station_lookup = stations.set_index('Number')['NAME']
    station_departures['start_station_name'] = station_departures['start_station_id'].map(station_lookup)

    return station_departures

#### Function to Normalize a Column

Using min-max normalization to score all of the values for downtime fraction, number of departures (U_i), and quantile-based concentration measures to the range [0,1]. Also inverting the quantile-based concentration measures using 1 - minmax(q_difference) because previously, a low q_difference value was desirable (most concentrated departure distribution), but inverting allows for a high value to indicate higher concentration. This ensures that 1 is the best value for all three measures, and 0 is the worst value. The min_max normalization for a set of data is as follows for a set $X$: $\frac{X - \min X}{\max X - \min X}$

In [22]:
def min_max(col):
    return (col - col.min()) / (col.max() - col.min())

#### Function to Get Normalized Score

In [23]:
def get_metrics_norm(downtime, e_bikes_by_start, station_quantiles):

    metrics = downtime.merge(e_bikes_by_start[['start_station_id','num_starts']], on='start_station_id', how='left')
    metrics = metrics.merge(station_quantiles[['quantile_difference']], on='start_station_id', how='left')
    metrics = metrics.drop_duplicates()

    metrics['starts_norm'] = min_max(metrics['num_starts'])
    metrics['downtime_norm'] = min_max(metrics['downtime_fraction'])
    metrics['quantile_norm'] = 1 - min_max(metrics['quantile_difference'])

    metrics['norm_score'] = metrics['starts_norm'] * metrics['downtime_norm'] * metrics['quantile_norm'] 
    metrics = metrics.sort_values(by='norm_score', ascending=False)

    return metrics

#### Function for Station Scoring Robustness Check -- input deltas & taus, output top 10 stations for each

Scoring uses the normalized number of departures, normalized and inverted quantile difference in time distribution (higher value $\implies$ more concentrated bike use in time), and normalized downtime fractions (percent of time effective for charging).

In [24]:
def score_stations(e_bikes, deltas, taus):
    
    all_top_stations = []

    for t in taus:
        for d in deltas:
            tau = t
            delta = d
            
            downtime = compute_downtime_fraction(e_bikes, delta, tau, station_lookup)
            downtime = downtime[['start_station_id', 'station_name', 'delta', 'tau', 'downtime_fraction']]

            station_quantiles = get_station_quantiles(e_bikes)

            ebike_departures = get_departure_info(e_bikes)
            
            metrics = get_metrics_norm(downtime, ebike_departures, station_quantiles)
            
            top10 = (metrics.groupby(['delta', 'tau'], as_index=False).head(10))

            all_top_stations.append(top10)

    all_top_stations = pd.concat(all_top_stations, ignore_index=True)

    return all_top_stations

#### Running Scoring

In [25]:
# DO NOT RUN AGAIN
# Saves the top 10 stations for each delta, tau pair at each minute cutoff

deltas = ['30min', '1h', '2h', '4h']
taus = [0,1,2]

minute_cutoffs = [0, 10, 20]  # ensure that trips included are at least (cutoff) minutes long

for cutoff in minute_cutoffs:   # check 'minutes_elapsed'
    e_bikes = ebikes[ebikes['minutes_elapsed'] >= cutoff].copy()
    e_bikes['started_at'] = pd.to_datetime(e_bikes['started_at'])  # Ensure format

    scores = score_stations(e_bikes, deltas, taus)
    #scores.to_csv(f'scores_cutoff_{cutoff}_mins.csv', index=False)

### Building Candidate Pool with Scores (this will get cleaned up I'm sorry)

From the Charging Time Score cells, the top 10 stations for each delta, tau pair at each minute cutoff are saved into 3 .csv files: 'scores_cutoff_{cutoff}_mins.csv'. To build a candidate pool for the best combinations of $k=3$ stations from each delta, tau, cutoff set. To rank the combinations, we can maximize the distance between the 3 chosen stations by choosing the greatest triangular perimeter: for stations $A, B, C$, find $d(A,B) + d(A,C) + d(B,C)$

#### Function to Get Pairwise Distance between Two Stations (input: tuple of station IDs)

In [26]:
def pairwise_dist(pair):

    lat0 = stations.loc[stations['Number'] == pair[0], 'Lat'].values[0]
    long0 = stations.loc[stations['Number'] == pair[0], 'Long'].values[0]
    lat1 = stations.loc[stations['Number'] == pair[1], 'Lat'].values[0]
    long1 = stations.loc[stations['Number'] == pair[1], 'Long'].values[0]

    return haversine(lat0, long0, lat1, long1)

#### Get Station Combos, Trips Between Ratio, and Perimeter Distance in One DF

In [27]:
def get_station_combo_info(ebikes_by_trip, ebikes_by_start, id_list, k, delta, tau, cutoff):
    results = []
    combos = list(combinations(id_list, k))

    trip_lookup = (ebikes_by_trip.set_index(['start_station_id', 'end_station_id'])['num_trips'])
    dep_lookup = (ebikes_by_start.set_index(['start_station_id'])['num_starts'])

    for combo in combos:
        perimeter = 0
        pairs = list(combinations(combo, 2))
        for pair in pairs:
            perimeter += pairwise_dist(pair)

        internal_trips = 0
        total_trips = 0

        for i in range(k):
            for j in range(k):
                if j != i:
                    internal_trips += trip_lookup.get((combo[i], combo[j]), 0)
            total_trips += dep_lookup.get(combo[i], 0)

        between_ratio = internal_trips / total_trips

        results.append((combo, perimeter, between_ratio))

    results_with_params = []
    for combo, perimeter, between_ratio in results:
        row = {f"station_{i+1}": combo[i] for i in range(k)}
        row["between_ratio"] = between_ratio
        row["perimeter"] = perimeter
        row["delta"] = delta
        row["tau"] = tau
        row["cutoff"] = cutoff
        results_with_params.append(row)

    return results_with_params


#### Save All Station Combos with Trip Between Ratios and Perimeters to Dataframe

In [28]:
minute_cutoffs = [0, 10, 20]                    # MAKE SURE THESE VALUE MATCH THE 'RUNNING SCORING' CELL
deltas = ['30min', '1h', '2h', '4h']
taus = [0,1,2]
k = 3
t = 5

all = []
for cutoff in minute_cutoffs:
    scores = pd.read_csv(f'scores_cutoff_{cutoff}_mins.csv')
    for delta in deltas:
        for tau in taus:

            id_list = list(scores.loc[(scores['delta'] == delta) & (scores['tau'] == tau), 'start_station_id'])
            all_combos = get_station_combo_info(ebikes_by_trip, ebikes_by_start, id_list, k, delta, tau, cutoff)
            all.extend(all_combos)
x = pd.DataFrame(all)


#### Normalize the Trip Between Ratios and Perimeters, Mutliply to Score each combo (1 = best, 0 = worst)

In [29]:
x['perimeter_norm'] = min_max(x['perimeter'])
x['trips_between_norm'] = 1 - min_max(x['between_ratio'])
x['combined_between_ratio_distance'] = x['perimeter_norm'] * x['trips_between_norm']

#### Get the Top 5 Combos by Normalized Score for each delta, tau, cutoff combo

In [30]:
top5 = (x.groupby(['delta', 'tau', 'cutoff'], sort=False, group_keys=False).apply(lambda df: df.sort_values('combined_between_ratio_distance',ascending=False).head(t)))

C:\Users\Julie\AppData\Local\Temp\ipykernel_16652\940110118.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top5 = (x.groupby(['delta', 'tau', 'cutoff'], sort=False, group_keys=False).apply(lambda df: df.sort_values('combined_between_ratio_distance',ascending=False).head(t)))


In [ ]:
candidates = pd.DataFrame(top5)

station_lookup = stations.set_index('Number')['NAME']
candidates['station1_name'] = candidates['station_1'].map(station_lookup)
candidates['station2_name'] = candidates['station_2'].map(station_lookup)
candidates['station3_name'] = candidates['station_3'].map(station_lookup)
#candidates.to_csv('candidate_list.csv', index=False)

PermissionError: [Errno 13] Permission denied: 'candidate_list.csv'

In [ ]:
c = pd.read_csv('candidate_list.csv')
c = c[['station_1', 'station_2', 'station_3']]

,station_1,station_2,station_3
0,A32004,B32007,M32017
1,A32004,C32034,M32017
2,D32046,B32007,M32017
3,B32007,D32024,M32017
4,B32007,M32061,M32017
...,...,...,...
175,A32002,B32002,C32036
176,M32011,A32002,C32036
177,M32011,B32002,C32036
178,A32002,C32036,M32005


In [45]:
row_sets_as_tuples = c.apply(lambda row: tuple(sorted(row.values)), axis=1)
frequency_counts = row_sets_as_tuples.value_counts()

final_stations = []
for station, count in frequency_counts.head(6).items():
    final_stations.append(station)

In [ ]:
counts = frequency_counts.reset_index()
counts.columns = ['stations', 'count']  # Turn frequency counts of each station combo into a dataframe

stations_df = counts['stations'].apply(pd.Series)  # Split station tuples into separate columns
stations_df.columns = ['station1', 'station2', 'station3']

final_df = pd.concat([stations_df, counts['count']], axis=1)  # Put stations (separated) and counts of appearance into a dataframe

final_df['station1_name'] = final_df['station1'].map(station_lookup)  # Add names of stations
final_df['station2_name'] = final_df['station2'].map(station_lookup)
final_df['station3_name'] = final_df['station3'].map(station_lookup)

In [59]:
final_df.head(10)

,station1,station2,station3,count,station1_name,station2_name,station3_name
0,B32002,C32036,M32011,11,Ruggles T Stop - Columbus Ave at Melnea Cass Blvd,Seaport Blvd at Sleeper St,Central Square at Mass Ave / Essex St
1,A32004,C32034,M32005,8,Longwood Ave at Binney St,Watermark Seaport - Boston Wharf Rd at Seaport...,MIT Stata Center at Vassar St / Main St
2,A32002,C32036,M32005,6,Commonwealth Ave at Agganis Way,Seaport Blvd at Sleeper St,MIT Stata Center at Vassar St / Main St
3,A32002,C32036,M32011,6,Commonwealth Ave at Agganis Way,Seaport Blvd at Sleeper St,Central Square at Mass Ave / Essex St
4,A32002,B32002,C32036,6,Commonwealth Ave at Agganis Way,Ruggles T Stop - Columbus Ave at Melnea Cass Blvd,Seaport Blvd at Sleeper St
5,A32004,C32034,M32037,6,Longwood Ave at Binney St,Watermark Seaport - Boston Wharf Rd at Seaport...,Ames St at Main St
6,A32010,B32002,M32011,5,South Station - 700 Atlantic Ave,Ruggles T Stop - Columbus Ave at Melnea Cass Blvd,Central Square at Mass Ave / Essex St
7,A32004,B32007,M32037,4,Longwood Ave at Binney St,Seaport Square - Seaport Blvd at Northern Ave,Ames St at Main St
8,A32002,C32036,M32037,4,Commonwealth Ave at Agganis Way,Seaport Blvd at Sleeper St,Ames St at Main St
9,A32004,B32007,M32005,4,Longwood Ave at Binney St,Seaport Square - Seaport Blvd at Northern Ave,MIT Stata Center at Vassar St / Main St


### K-Medoids (this will also get cleaned up)

In [60]:
import kmedoids

In [62]:
k = 3
result = kmedoids.fasterpam(dist_matrix, k)

In [83]:
idx_to_id = {idx: station_id for station_id, idx in id_to_idx.items()}

kmedoid_stations = []

for i in range(len(result.medoids)):
    kmedoid_stations.append(idx_to_id[result.medoids[i]])

print(kmedoid_stations)
for id in kmedoid_stations:
    print(stations.loc[stations['Number'] == id, 'NAME'])

build_map(kmedoid_stations)

['A32021', 'M32043', 'B32027']
221    Nashua Street at Red Auerbach Way [Extension]
Name: NAME, dtype: object
229    Mt Auburn
Name: NAME, dtype: object
220    NCAAA - Walnut Ave at Crawford St
Name: NAME, dtype: object
